In [1]:
pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,classification_report
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import numpy as np
import logging

In [3]:
df_treino = pd.read_csv('Fake.br-Corpus_English.csv',sep=',')
df_treino['label'] = df_treino['label'].apply(lambda x: 0 if x == "fake" else 1)

In [4]:
df_treino.drop(columns=['index'],inplace = True)
df_treino.rename(columns={'preprocessed_news': 'text'}, inplace = True)
df_treino.rename(columns={'label': 'labels'}, inplace = True)
df_treino = df_treino[['text', 'labels']]
df_treino.head()

,text,labels
0,katia abreu says will put expulsion frame do n...,0
1,ray peita bolsonaro conservative fake intervie...,0
2,reinaldo azevedo unmasked federal police staun...,0
3,frightening report bndes shows public money gu...,0
4,american broadcaster talks about pt selling il...,0


In [5]:
df_treino=df_treino.iloc[np.random.permutation(len(df_treino))].reset_index(drop=True)
df_treino

,text,labels
0,protesters protest goiania against lula politi...,1
1,dilma almost dismissed army commander declared...,0
2,tiririca rips verb fear was wrong to pay i don...,0
3,ethics council postpone decision on cassation ...,1
4,petrobras proposes payment us bi close action ...,1
...,...,...
7195,president pt praises decision gave freedom bru...,0
7196,life supreme judge sweden privileged forum imm...,0
7197,stf decision rekindles lula pt after supreme g...,1
7198,fear hazelnut cream plateau decides to cancel ...,0


In [6]:
import torch
device = torch.device("cuda:0" if (torch.cuda.is_available())else "cpu")
device

device(type='cuda', index=0)

In [7]:
kf = KFold(n_splits=10)

In [8]:
results = []

model_args = ClassificationArgs()
model_args.overwrite_output_dir=True
model_args.num_train_epochs = 1
model_args.num_labels = 2
model_args.silent = True

logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)
model = ClassificationModel('bert', 'bert-base-uncased', args=model_args, use_cuda=device)

for train_index, val_index in kf.split(df_treino):  
    train_df = df_treino.iloc[train_index]
    val_df = df_treino.iloc[val_index]
    model.train_model(train_df)
    
    result, model_output, wrong_predictions = model.eval_model(val_df, acc=accuracy_score)     
 
    predictions, raw_outputs  = model.predict(val_df['text'].values.tolist())
    print(classification_report(predictions,val_df["labels"],digits=4))
    print(f"Accuracy: {result['acc']}")

    results.append(result['acc'])

              precision    recall  f1-score   support

           0     0.9229    0.9124    0.9176       354
           1     0.9162    0.9262    0.9212       366

    accuracy                         0.9194       720
   macro avg     0.9195    0.9193    0.9194       720
weighted avg     0.9195    0.9194    0.9194       720

Accuracy: 0.9194444444444444
              precision    recall  f1-score   support

           0     0.9653    0.9605    0.9629       405
           1     0.9495    0.9556    0.9525       315

    accuracy                         0.9583       720
   macro avg     0.9574    0.9580    0.9577       720
weighted avg     0.9584    0.9583    0.9583       720

Accuracy: 0.9583333333333334
              precision    recall  f1-score   support

           0     0.9697    0.9591    0.9644       367
           1     0.9580    0.9688    0.9634       353

    accuracy                         0.9639       720
   macro avg     0.9638    0.9640    0.9639       720
weighted avg    